In [2]:
import torch
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

df=pd.read_excel('final_210429_tous labels.xlsx',engine='openpyxl')
df.head()

,Messages,Nom du fichier,Date de création,Date de modification,catégorie
0,"Bonjour,Ci-joint le mandat sepa et le rib.Cord...",AAMkAGRiZjFhYjc3LTM2YTctNDE3MC1hODdmLTZhMTEwYm...,Fri Apr 2 11:22:49 2021,Fri Aug 28 14:20:41 2020,contrat
1,DANIEL GUEGANSOCIETE DES TRANSPORTS GUEGANCl...,AAMkAGRiZjFhYjc3LTM2YTctNDE3MC1hODdmLTZhMTEwYm...,Fri Apr 2 11:22:49 2021,Fri Aug 28 14:20:37 2020,commande de badge
2,"Bonjour, J’ai essayé de faire la modification...",AAMkAGRiZjFhYjc3LTM2YTctNDE3MC1hODdmLTZhMTEwYm...,Fri Apr 2 11:22:49 2021,Mon Aug 31 08:28:36 2020,contrat
3,"Bonjour, merci de me faire un badge avec la gé...",AAMkAGRiZjFhYjc3LTM2YTctNDE3MC1hODdmLTZhMTEwYm...,Fri Apr 2 11:22:49 2021,Wed Aug 19 15:52:02 2020,commande de badge
4,No Message,AAMkAGRiZjFhYjc3LTM2YTctNDE3MC1hODdmLTZhMTEwYm...,Fri Apr 2 11:22:49 2021,Wed Aug 5 11:18:33 2020,no message


In [8]:
df = df.drop(['Date de création',  'Date de modification','Nom du fichier'], axis=1)

In [9]:
df['catégorie'].value_counts()

contrat                           806
commande de badge                 533
no message                        485
demande de renseignement          277
renseignement facture             142
interne Axxès                      87
remplacement badge                 86
réclamation qualité de service     72
réclamation facture                70
réclamation logistique             34
no                                  4
amende Belgique                     2
Name: catégorie, dtype: int64

In [10]:
possible_labels = df.catégorie.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'contrat': 0,
 'commande de badge': 1,
 'no message': 2,
 'réclamation qualité de service': 3,
 'interne Axxès': 4,
 'renseignement facture': 5,
 'demande de renseignement': 6,
 'réclamation facture': 7,
 'remplacement badge': 8,
 'réclamation logistique': 9,
 'amende Belgique': 10,
 'no': 11,
 nan: 12}

In [11]:
df['label'] = df.catégorie.replace(label_dict)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['catégorie', 'label', 'data_type']).count()

Messages
catégorie                      label data_type          
amende Belgique                10    train             2
commande de badge              1     train           453
                                     val              80
contrat                        0     train           684
                                     val             121
demande de renseignement       6     train           235
                                     val              42
interne Axxès                  4     train            74
                                     val              13
no                             11    train             3
                                     val               1
no message                     2     train           406
                                     val              73
remplacement badge             8     train            73
                                     val              13
renseignement facture          5     train           121
                                     val              21
réclamation facture            7     train            60
                                     val              10
réclamation logistique         9     train            29
                                     val               5
réclamation qualité de service 3     train            61
                                     val              11

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Messages.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Messages.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


ValueError: Input is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

Loading and Evaluating the Model

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)